# Runnables 구조 검토

LCEL로 `runnable` 을 생성한 후에는 종종 이를 검사하여 어떤 일이 일어나고 있는지 더 잘 파악하고 싶을 것입니다.

이 노트북에서는 이를 수행하는 몇 가지 방법을 다룹니다.


In [ ]:
%pip install -qU langchain langchain-openai faiss-cpu tiktoken

In [ ]:
# 그래프를 그리기 위한 라이브러리 설치
%pip install -qU grandalf

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    # 텍스트 데이터로부터 FAISS 벡터 저장소를 생성합니다.
    ["Teddy is an AI engineer who loves programming!"],
    embedding=OpenAIEmbeddings(),
)

# 벡터 저장소를 기반으로 retriever를 생성합니다.
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}  

Question: {question}"""

prompt = ChatPromptTemplate.from_template(
    template
)  # 템플릿을 기반으로 ChatPromptTemplate을 생성합니다.

model = ChatOpenAI()  # ChatOpenAI 모델을 초기화합니다.

# chain 을 생성합니다.
chain = (
    # 검색 컨텍스트와 질문을 지정합니다.
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt  # 프롬프트를 생성합니다.
    | model  # 언어 모델을 실행합니다.
    | StrOutputParser()  # 출력 결과를 문자열로 파싱합니다.
)

## 그래프 구성 확인

runnable의 그래프를 얻을 수 있습니다.


`chain.get_graph()` 메서드는 체인의 실행 그래프를 반환합니다.

- 이 메서드는 체인의 각 노드와 노드 간의 연결을 나타내는 그래프 객체를 반환합니다.
- 그래프의 노드는 체인의 각 단계를 나타내며, 에지(edge)는 단계 간의 데이터 흐름을 나타냅니다.


In [19]:
# 체인의 그래프에서 노드를 가져옵니다.
chain.get_graph().nodes

{'46d23d1b25d6485daf44722926e09252': Node(id='46d23d1b25d6485daf44722926e09252', data=<class 'pydantic.v1.main.RunnableParallel<context,question>Input'>),
 '1d02513a446d49f9bd47bcc35a5b1a50': Node(id='1d02513a446d49f9bd47bcc35a5b1a50', data=<class 'pydantic.v1.main.RunnableParallel<context,question>Output'>),
 'a19608744b9642efaab6b7d733f8589f': Node(id='a19608744b9642efaab6b7d733f8589f', data=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x17346e920>)),
 '98f3a9f1ad46441f9608f67d427f0f97': Node(id='98f3a9f1ad46441f9608f67d427f0f97', data=RunnablePassthrough()),
 '989023b7f06f4755b13db7344ed30239': Node(id='989023b7f06f4755b13db7344ed30239', data=ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the question based only on the following context:\n{context}  \n\nQuestion: {question}'))

In [20]:
# 체인의 그래프에서 엣지를 가져옵니다.
chain.get_graph().edges

[Edge(source='565136c5356541828f1879296c6ba0d6', target='5810102acbf84bdeb6342cc5b3c54a30', data=None),
 Edge(source='5810102acbf84bdeb6342cc5b3c54a30', target='ecefd39e4d62439189527d8c424923e9', data=None),
 Edge(source='565136c5356541828f1879296c6ba0d6', target='4823ded9b1d3429992634ea5ec76a67b', data=None),
 Edge(source='4823ded9b1d3429992634ea5ec76a67b', target='ecefd39e4d62439189527d8c424923e9', data=None),
 Edge(source='ecefd39e4d62439189527d8c424923e9', target='f599128d839747678f3193eea303542b', data=None),
 Edge(source='f599128d839747678f3193eea303542b', target='bceabc5b2ba74ed5a59793cdf94e880a', data=None),
 Edge(source='bb73629324bb4138985a78eb606e4071', target='a6d9dec55c6540eeb4b47426bd4aaa6f', data=None),
 Edge(source='bceabc5b2ba74ed5a59793cdf94e880a', target='bb73629324bb4138985a78eb606e4071', data=None)]

## 그래프 출력

그래프를 출력하면 이해하기 쉬운 형태로 표현할 수 있습니다.

비록 출력 결과가 매우 읽기 쉽지는 않지만, 출력을 통해 보다 이해하기 쉬운 형태로 그래프를 확인할 수 있습니다.


In [11]:
# 체인의 그래프를 ASCII 형식으로 출력합니다.
chain.get_graph().print_ascii()

           +---------------------------------+         
           | Parallel<context,question>Input |         
           +---------------------------------+         
                    **               **                
                 ***                   ***             
               **                         **           
+----------------------+              +-------------+  
| VectorStoreRetriever |              | Passthrough |  
+----------------------+              +-------------+  
                    **               **                
                      ***         ***                  
                         **     **                     
           +----------------------------------+        
           | Parallel<context,question>Output |        
           +----------------------------------+        
                             *                         
                             *                         
                             *                  

## 프롬프트 가져오기

체인에서 중요한 부분은 사용되는 프롬프트입니다.


`chain.get_prompts()` 메서드는 체인에서 사용되는 프롬프트(prompt) 객체의 리스트를 반환합니다.


In [15]:
chain.get_prompts()  # 체인에서 사용되는 프롬프트를 가져옵니다.

[ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the question based only on the following context:\n{context}  \n\nQuestion: {question}'))])]